In [1]:
import requests
import time
import io
import os
import re 
import pandas as pd
import jiwer
from pathlib import Path
from PIL import Image
from pdf2image import convert_from_bytes
from IPython.display import display, Markdown, HTML
from tqdm.notebook import tqdm

PDF_LETTER_DIR = Path("input_pdfs-Letter")
PDF_BOARD_DIR = Path("input_pdfs-Board")
GROUND_TRUTH_DIR = Path("ground_truth")
# GROUND_TRUTH_DIR = Path("ground_truth-Letter")
# GROUND_TRUTH_DIR = Path("ground_truth-Board")
TYPHOON_OCR_ENDPOINT = "http://typhoon-ocr:8000/process"

In [2]:
OCR_CORRECTION_MAP = {
    "นศ.สรท.":"นศ.สธท.", "ศชบ":"ศซบ", "กวถ.ศชบ.ทหาร": "กวก.ศซบ.ทหาร", "กรก.ศชบ.ทหาร": "กรก.ศซบ.ทหาร",
    "กธก.ศชบ.ทหาร": "กธก.ศซบ.ทหาร", "กวก.ศชบ.ทหาร": "กวก.ศซบ.ทหาร", "กหค.ศทท.สส.ทหาร": "กทค.ศทท.สส.ทหาร",
    "กปภ.ศชบ.ทหาร": "กปก.ศซบ.ทหาร", "กก.กธก.ศชบ.ทหาร": "หก.กธก.ศซบ.ทหาร", "กวภ.ศชบ.ทหาร": "กวก.ศซบ.ทหาร",
    "ศช.ทหาร. ": "ศซบ.ทหาร ", "คุณท.๖๗": "คกนท.๖๗", "สน.พน.วสท.สปท.": "สน.ผบ.วสท.สปท.",
    "สน.พบ.สปท.": "สน.ผบ.สปท.", "กวต.ศชบ.ทหาร": "กวก.ศซบ.ทหาร", "นชต.ศชบ.ทหาร": "นขต.ศซบ.ทหาร",
    "ผอ.ศชบ.ทหาร": "ผอ.ศซบ.ทหาร", "ศชย.สปท. ": "ศศย.สปท. ", "รอง ผอ.กพศ.ศชย.สปท.": "รอง ผอ.กภศ.ศศย.สปท.",
    "สบ.บก.ทท. ": "สน.บก.บก.ทท. ", "ยน.ทหาร": "ยบ.ทหาร", "บก.ทหาร": "บก.ทท.", "สสค.บก.ทท.": "สลก.บก.ทท.",
    "สสภ.ทหาร": "สสก.ทหาร", "ชว.ทหาร": "ขว.ทหาร", "นทฟ. ": "นทพ.", "กวภ.ศช.น.ทหาร": "กวก.ศซบ.ทหาร",
    "ศช.บ.ทหาร": "ศซบ.ทหาร", "กกล.นชช.ทหาร": "กกล.นซบ.ทหาร", "นชช.ทหาร": "นซบ.ทหาร",
    "ศชล.นชช.ทหาร": "ศซล.นซบ.ทหาร", "กปช.ศชบ.สสท.ทร. ": "กปซ.ศซบ.สสท.ทร.","ถวก.ศชล.นซบ.ทหาร": "กวก.ศซล.นซบ.ทหาร", 
    "กศช.สศท.สปท.": "กศษ.สศท.สปท.", "เสร.สปท.": "เสธ.สปท.", "นชบ.ทหาร": "นซบ.ทหาร",
    "รธ.ชน.ทหาร": "รร.ซบ.ทหาร", "สน.ทหาร": "สบ.ทหาร", "กสม.สน.ทหาร. ": "กสบ.สบ.ทหาร. ",
    "นชต.ศช.ทหาร": "นขต.นซบ.ทหาร", "กวจ.ศชน.ทหาร": "กวก.ศซบ.ทหาร", "ผอ.ศช.ปทหาร": "ผอ.ศซบ.ทหาร",
    "กน.ทหาร": "กบ.ทหาร", "ศตถ. ": "ศตก. ", "สคท.สปท.": "สศท.สปท.", "กรภ.ศชบ.ทหาร": "กธก.ศซบ.ทหาร",
    "รร.รปภ.ศธ. ": "รร.รปภ.ศรภ.", "นทท.": "นทพ.", "กรมทหาร": "กร.ทหาร", "คชช.ทหาร": "ศซบ.ทหาร",
    "ถนนผจงพหาร": "กนผ.กร.ทหาร", "สวผ.ยก.ทหาร": "สวฝ.ยก.ทหาร", "กหศ.ศสภ.ยก.ทหาร": "กฝศ.ศสภ.ยก.ทหาร",
    "กหม.นก.สปท.": "กทด.บก.สปท.", "เลขา.สปท.": "เสธ.สปท.", "ผอ.บทว.สปท.": "ผอ.บฑว.สปท.",
    "กจก.สนส. กม.ทหาร": "กจก.สบส.กบ.ทหาร", "กสม.สน.ทหาร": "กสบ.สบ.ทหาร", "กพศ.ศสภ.ยก.ทหาร": "กฝศ.ศสภ.ยก.ทหาร",
    "ถนนผ.กร.ทหาร": "กนผ.กร.ทหาร", "ศชบ.ทอ.": "ศซบ.ทอ.", "รร.รปภ.ศธ.": "รร.รปภ.ศรภ.", "กคช.บก.นทพ.": "กกช.บก.นทพ.",
    "กบ.สคร.กร.ทหาร": "กบภ.สกร.กร.ทหาร", "กห.อต๊อด.๑๐.๑":"กห ๐๓๐๑.๑๐.๑","จึงเสนอมามาเพื่อกรุณาพิจารณา":"จึงเสนอมาเพื่อกรุณาพิจารณา",
    "๕๗๖๓๙(๔๗).":"๕๗๒๑๗๔๗).","๐-๒๕๗๒.๑๗๔๗.":"๐ ๒๕๗๒ ๑๗๔๗","กห.อต๊อก.๑๐.๑":"กห ๐๓๐๑.๑๐.๑","กปภ.๓":"กปก.๓","กธถ.ศชบ.ทหาร":"กธก.ศซบ.ทหาร",
    "กรก.ศชบ.ทหาร":"กธก.ศซบ.ทหาร","ผช.ผอ.กรก.ศชบ.ทหาร":"ผช.ผอ.กรภ.ศซบ.ทหาร","กท.อต๊อก.๑":"กห ๐๓๐๑.๑๐.๑","ผช.ผอ.กรก.ศชบ.ทหาร":"ผช.ผอ.กรภ.ศซบ.ทหาร",
    "กก.กธก.ศซบ.ทหาร":"หก.กธก.ศซบ.ทหาร","กกล.นชบ.ทหาร":"กกล.นซบ.ทหาร","๐.๒๒๗๕.๕๗๑๖":"๐ ๒๒๗๕ ๕๗๑๖","อิลล์":"ฮิลส์","ไม่กำหนดชื่อ":"ไม่กำหนดชั้นยศ",
    "ผอ.กพศ.ศคย.สปท.":"ผอ.กภศ.ศศย.สปท.","ผอ.ศคย.สปท.":"ผอ.ศศย.สปท.", "..สสท.ทร.(ศซบ.โทร.๕๗๘๙)":"สสท.ทร. (ศซบ. โทร.๕๗๘๙๐)",
    "คานฑ์.๖๗":"คกนท.๖๗","สน.พ.วสท.สปท.":"สน.ผบ.วสท.สปท.","สน.ผ.สปท.":"สน.ผบ.สปท.","ผอ.กพ.วสท.สปท.":"ผอ.กพผ.วสท.สปท.",
    "นายทหารอุบมิติข่าว":"นายทหารอนุมัติข่าว","กระดาษเชิญข่าวร่วม (ทท.)":"กระดาษเขียนข่าวร่วม (ทท.)",
    "จึงเสนอมาระบกวนโปรด":"จึงเสนอมาเพื่อโปรด", "ลาอฉก.": "ลาออก", "0":"๐", "1":"๑", "2":"๒", "3":"๓", "4":"๔", "5":"๕", "6":"๖", "7":"๗", "8":"๘", "9":"๙"
}

def post_process_ocr(ocr_text: str) -> str:
    if not ocr_text or not isinstance(ocr_text, str):
        return ""
    processed_text = ocr_text
    sorted_correction_keys = sorted(OCR_CORRECTION_MAP.keys(), key=len, reverse=True)
    for wrong_word_key in sorted_correction_keys:
        correct_word_value = OCR_CORRECTION_MAP[wrong_word_key]
        processed_text = processed_text.replace(wrong_word_key, correct_word_value)
        processed_text = re.sub(r'\s+', ' ', processed_text).strip()
        processed_text = re.sub(r'\s+\.', '.', processed_text)
        processed_text = processed_text.replace('\n', ' ')
        processed_text = processed_text.replace("“", "\"").replace("”", "\"")
        processed_text = processed_text.replace("‘", "'").replace("’", "'")
        processed_text = re.sub(r'\-{3,}', '', processed_text)
        processed_text = re.sub(r'\*{2,}', '', processed_text)
        processed_text = re.sub(r'\s- ', '', processed_text)
        processed_text = processed_text.replace('#', '')
        processed_text = processed_text.replace('|', '')
    return processed_text

def pdf_to_images(pdf_bytes, dpi=300):
    try:
        return convert_from_bytes(pdf_bytes, dpi=dpi)
    except Exception as e:
        print(f"เกิดข้อผิดพลาดในการแปลง PDF: {e}")
        return []

def run_typhoon_ocr(pdf_path: Path):
    try:
        with open(pdf_path, "rb") as f:
            pdf_bytes = f.read()
    except FileNotFoundError:
        return {"error": f"ไม่พบไฟล์: {pdf_path}"}, 0

    images = pdf_to_images(pdf_bytes)
    if not images:
        return {"error": "การแปลง PDF เป็นรูปภาพล้มเหลว"}, 0

    full_text = []
    start_time = time.time()
    for i, img in enumerate(images):
        img_byte_arr = io.BytesIO()
        img.save(img_byte_arr, format='PNG')
        files = {'file': (f'page_{i+1}.png', img_byte_arr.getvalue(), 'image/png')}
        try:
            response = requests.post(TYPHOON_OCR_ENDPOINT, files=files, timeout=180)
            response.raise_for_status()
            ocr_result = response.json()
            
            result_text = ocr_result.get("result")
            if result_text and isinstance(result_text, str) and result_text.strip():
                full_text.append(result_text.strip())
            else:
                error_msg = f"[ERROR on Page {i+1}: API did not return valid text in 'result' key. Response: {ocr_result}]"
                print(f"     - {error_msg}")
                full_text.append(error_msg) 

        except requests.exceptions.RequestException as e:
            return {"error": f"API Connection Error: {e}"}, 0
        except ValueError:
            return {"error": f"API did not return valid JSON. Response: {response.text}"}, 0
            
    exec_time = time.time() - start_time
    separator = "\n\n" + "="*20 + " END OF PAGE " + "="*20 + "\n\n"
    final_text = separator.join(full_text)
    
    return {"text": final_text, "page_count": len(images)}, exec_time


def calculate_metrics(hypothesis: str, reference: str):
    if not reference or not reference.strip():
        return {"wer": None, "cer": None}
    
    transformation = jiwer.Compose([
        jiwer.ToLowerCase(),
        jiwer.RemoveMultipleSpaces(),
        jiwer.Strip(),
        jiwer.RemovePunctuation(),
    ])
    
    try:
        reference_clean = transformation(reference)
        hypothesis_clean = transformation(hypothesis)
        
        wer = jiwer.wer(reference_clean, hypothesis_clean)
        cer = jiwer.cer(reference_clean, hypothesis_clean)
        return {"wer": wer, "cer": cer}
        
    except Exception as e:
        return {"wer": None, "cer": None}

def safe_percent_format(val):
    if isinstance(val, (int, float)) and pd.notna(val):
        return f"{val:.2%}"
    return "N/A"

## Internal Letter

In [3]:
# Focus on Raw OCR-Internal Letter

pdf_files = sorted(list(PDF_LETTER_DIR.glob("*.pdf")))

if not pdf_files:
    display(Markdown(f"<font color='red'>**ไม่พบไฟล์ PDF ในโฟลเดอร์ `{PDF_LETTER_DIR}` กรุณาตรวจสอบว่า Path ถูกต้อง**</font>"))
else:
    display(Markdown(f"### 🚀 เริ่มการทดสอบระบบ OCR กับไฟล์ทั้งหมด {len(pdf_files)} ไฟล์"))
    
    results_list = []
    
    for pdf_path in tqdm(pdf_files, desc="กำลังประมวลผลไฟล์"):

        # run OCR
        ocr_result, exec_time = run_typhoon_ocr(pdf_path)
        raw_ocr_text = ocr_result.get("text", "")
        
        # load Ground Truth
        pdf_stem = pdf_path.stem
        gt_path = GROUND_TRUTH_DIR / f"{pdf_stem}.txt"
        print(f"   - PDF Stem: '{pdf_stem}'")
        print(f"   - กำลังมองหา Ground Truth ที่: '{gt_path}'")
        
        ground_truth_text = ""
        if gt_path.is_file():
            print(f"   - ✅ พบไฟล์ Ground Truth!")
            try:
                ground_truth_text = gt_path.read_text(encoding="utf-8").strip()
                if ground_truth_text:
                    print(f"     - SUCCESS: อ่านเนื้อหาสำเร็จ มี {len(ground_truth_text)} ตัวอักษร")
                else:
                    print(f"     - WARNING: อ่านไฟล์สำเร็จ แต่ไฟล์ว่างเปล่า!")
            except Exception as e:
                print(f"     - ❌ ERROR: เกิดข้อผิดพลาดในการอ่านไฟล์: {e}")
        else:
            print(f"   - ❌ ไม่พบไฟล์ Ground Truth")
        
        # Metrics Calculation
        metrics = calculate_metrics(raw_ocr_text, ground_truth_text)

        results_list.append({
            "File": pdf_path.name,
            "Time (s)": exec_time,
            "WER": metrics.get('wer'), 
            "CER": metrics.get('cer'), 
            "Raw OCR Text": raw_ocr_text,
            "Ground Truth Text": ground_truth_text 
        })
        

    df_results = pd.DataFrame(results_list)

    df_results["WAcc"] = 1 - df_results["WER"].fillna(1) # ถ้า WER เป็น None ให้ถือว่า Error 100%
    df_results["CAcc"] = 1 - df_results["CER"].fillna(1) # ถ้า CER เป็น None ให้ถือว่า Error 100%
    display(Markdown(f"<hr>"))
    display(Markdown(f"### 📊 ตารางสรุปผลการทดสอบ (Raw OCR Accuracy)"))

    display_cols = ["File", "Time (s)", "CER", "WER", "CAcc", "WAcc"]

    def safe_percent_format(val):
        if isinstance(val, (int, float)) and pd.notna(val):
            return f"{val:.2%}"
        return "N/A"

    has_numeric_metrics = pd.to_numeric(df_results["CER"], errors='coerce').notna().any()

    styler = df_results[display_cols].style.format({
        "Time (s)": "{:.2f}",
        "CER": safe_percent_format,
        "WER": safe_percent_format,
        "CAcc": safe_percent_format,
        "WAcc": safe_percent_format,
    }).set_properties(**{'text-align': 'left'}).set_table_styles(
        [dict(selector='th', props=[('text-align', 'left')])]
    )

    if has_numeric_metrics:
        styler.background_gradient(cmap='RdYlGn_r', subset=["CER", "WER"])
        styler.background_gradient(cmap='YlGn', subset=["CAcc", "WAcc"])
    
    display(styler)
    display(Markdown(f"### 📈 ค่าเฉลี่ยรวม"))
    
    avg_metrics = df_results[["Time (s)", "CAcc", "WAcc"]].mean()

    display(Markdown(f"- **เวลาประมวลผลเฉลี่ย:** `{avg_metrics['Time (s)']:.2f}` วินาที/ไฟล์"))
    if pd.notna(avg_metrics['CAcc']):
        display(Markdown(f"- **Character Accuracy (CAcc) เฉลี่ย:** `{avg_metrics['CAcc']:.2%}`"))
        display(Markdown(f"- **Word Accuracy (WAcc) เฉลี่ย:** `{avg_metrics['WAcc']:.2%}`"))
    else:
        display(Markdown(f"- **Accuracy:** `N/A (ไม่มีข้อมูล Ground Truth)`"))

    # OCR Result Display
    display(Markdown(f"<hr>"))
    display(Markdown(f"### 📝 ผลลัพธ์เปรียบเทียบ (Raw OCR vs Ground Truth)"))
    
    for _, row in df_results.iterrows():
        display(Markdown(f"---"))
        display(Markdown(f"#### 📄 **ไฟล์:** `{row['File']}`"))

        html_output = f"""
        <details>
            <summary>คลิกเพื่อดูผลลัพธ์ Raw OCR</summary>
            <div style="background-color:#f7f7f7; border:1px solid #ddd; padding:10px; margin-top:10px; white-space: pre-wrap; word-wrap: break-word;">{row['Raw OCR Text']}</div>
        </details>
        <details>
            <summary>คลิกเพื่อดู Ground Truth</summary>
            <div style="background-color:#e6f7ff; border:1px solid #b3e0ff; padding:10px; margin-top:5px; white-space: pre-wrap; word-wrap: break-word;">{row['Ground Truth Text']}</div>
        </details>
        """
        display(HTML(html_output))

### 🚀 เริ่มการทดสอบระบบ OCR กับไฟล์ทั้งหมด 10 ไฟล์

กำลังประมวลผลไฟล์:   0%|          | 0/10 [00:00<?, ?it/s]

   - PDF Stem: '000-2'
   - กำลังมองหา Ground Truth ที่: 'ground_truth/000-2.txt'
   - ✅ พบไฟล์ Ground Truth!
     - SUCCESS: อ่านเนื้อหาสำเร็จ มี 869 ตัวอักษร
   - PDF Stem: '001-2'
   - กำลังมองหา Ground Truth ที่: 'ground_truth/001-2.txt'
   - ✅ พบไฟล์ Ground Truth!
     - SUCCESS: อ่านเนื้อหาสำเร็จ มี 773 ตัวอักษร
   - PDF Stem: '003-2'
   - กำลังมองหา Ground Truth ที่: 'ground_truth/003-2.txt'
   - ✅ พบไฟล์ Ground Truth!
     - SUCCESS: อ่านเนื้อหาสำเร็จ มี 1127 ตัวอักษร
   - PDF Stem: '005-2'
   - กำลังมองหา Ground Truth ที่: 'ground_truth/005-2.txt'
   - ✅ พบไฟล์ Ground Truth!
     - SUCCESS: อ่านเนื้อหาสำเร็จ มี 1338 ตัวอักษร
   - PDF Stem: '008-2'
   - กำลังมองหา Ground Truth ที่: 'ground_truth/008-2.txt'
   - ✅ พบไฟล์ Ground Truth!
     - SUCCESS: อ่านเนื้อหาสำเร็จ มี 846 ตัวอักษร
   - PDF Stem: '009-2'
   - กำลังมองหา Ground Truth ที่: 'ground_truth/009-2.txt'
   - ✅ พบไฟล์ Ground Truth!
     - SUCCESS: อ่านเนื้อหาสำเร็จ มี 1836 ตัวอักษร
   - PDF Stem: '010-2'
   - กำลังมองห

<hr>

### 📊 ตารางสรุปผลการทดสอบ (Raw OCR Accuracy)

...

#### 📄 **ไฟล์:** `012-2.pdf`

---

#### 📄 **ไฟล์:** `013-2.pdf`

---

#### 📄 **ไฟล์:** `015-2.pdf`

In [4]:
# Focus on Processed OCR-Internal Letter

pdf_files = sorted(list(PDF_LETTER_DIR.glob("*.pdf")))

if not pdf_files:
    display(Markdown(f"<font color='red'>**ไม่พบไฟล์ PDF ในโฟลเดอร์ `{PDF_LETTER_DIR}` กรุณาตรวจสอบว่า Path ถูกต้อง**</font>"))
else:
    display(Markdown(f"### 🚀 เริ่มการทดสอบระบบ OCR กับไฟล์ทั้งหมด {len(pdf_files)} ไฟล์"))
    
    results_list = []
    
    for pdf_path in tqdm(pdf_files, desc="กำลังประมวลผลไฟล์"):

        # run OCR
        ocr_result, exec_time = run_typhoon_ocr(pdf_path)
        raw_ocr_text = ocr_result.get("text", "")
        
        # do Post-processing
        processed_ocr_text = post_process_ocr(raw_ocr_text)
        
        # load Ground truth
        pdf_stem = pdf_path.stem
        gt_path = GROUND_TRUTH_DIR / f"{pdf_stem}.txt"
        
        ground_truth_text = ""
        if gt_path.is_file(): 
            print(f"   - ✅ พบไฟล์ Ground Truth!")
            try:
                try:
                    ground_truth_text = gt_path.read_text(encoding="utf-8").strip()
                except UnicodeDecodeError:
                    print("     - WARNING: ไม่สามารถอ่านแบบ utf-8 ได้ ลองอ่านแบบ TIS-620")
                    ground_truth_text = gt_path.read_text(encoding="tis-620").strip()

                if ground_truth_text:
                    print(f"     - SUCCESS: อ่านเนื้อหาสำเร็จ มี {len(ground_truth_text)} ตัวอักษร")
                else:
                    print(f"     - WARNING: อ่านไฟล์สำเร็จ แต่ไฟล์ว่างเปล่า!")
            except Exception as e:
                print(f"     - ❌ ERROR: เกิดข้อผิดพลาดในการอ่านไฟล์: {e}")
        else:
            print(f"   - ❌ ไม่พบไฟล์ Ground Truth หรือไม่ใช่ไฟล์ปกติ (อาจเป็นโฟลเดอร์?)")
        
        # Metrics calculation
        metrics_before = calculate_metrics(raw_ocr_text, ground_truth_text)
        metrics_after = calculate_metrics(processed_ocr_text, ground_truth_text)
        
        results_list.append({
            "File": pdf_path.name,
            "Time (s)": exec_time,
            "WER (Before)": metrics_before['wer'],
            "CER (Before)": metrics_before['cer'],
            "WER (After)": metrics_after['wer'],
            "CER (After)": metrics_after['cer'],
            "Raw OCR Text": raw_ocr_text,
            "Processed OCR Text": processed_ocr_text,
            "Ground Truth Text": ground_truth_text
        })
        
    df_results = pd.DataFrame(results_list)
    display(Markdown(f"<hr>"))
    display(Markdown(f"### 📊 ตารางสรุปผลการทดสอบ"))
    df_results["WAcc (After)"] = 1 - df_results["WER (After)"].fillna(0)
    df_results["CAcc (After)"] = 1 - df_results["CER (After)"].fillna(0)

    display_cols = [
        "File", "Time (s)", 
        "CER (After)", "WER (After)", 
        "CAcc (After)", "WAcc (After)"
    ]

    has_numeric_metrics = pd.to_numeric(df_results["CER (After)"], errors='coerce').notna().any()

    # Format Data 
    styler = df_results[display_cols].style.format({
        "Time (s)": "{:.2f}",
        "CER (After)": safe_percent_format,
        "WER (After)": safe_percent_format,
        "CAcc (After)": safe_percent_format, 
        "WAcc (After)": safe_percent_format, 
    }).background_gradient(
        cmap='RdYlGn_r', 
        subset=["CER (After)", "WER (After)"]
    ).background_gradient(
        cmap='YlGn', 
        subset=["CAcc (After)", "WAcc (After)"]
    ).set_properties(**{'text-align': 'left'}).set_table_styles(
        [dict(selector='th', props=[('text-align', 'left')])]
    )

    display(styler)
    display(Markdown(f"### 📈 ค่าเฉลี่ยรวม"))

    avg_metrics = df_results[display_cols[1:]].mean() 
    display(Markdown(f"- **เวลาประมวลผลเฉลี่ย:** `{avg_metrics['Time (s)']:.2f}` วินาที/ไฟล์"))

    if pd.notna(avg_metrics['CAcc (After)']):
        display(Markdown(f"- **Character Accuracy (CAcc) เฉลี่ย:** `{avg_metrics['CAcc (After)']:.2%}` (ความถูกต้องระดับตัวอักษร)"))
        display(Markdown(f"- **Word Accuracy (WAcc) เฉลี่ย:** `{avg_metrics['WAcc (After)']:.2%}` (ความถูกต้องระดับคำ)"))
    else:
        display(Markdown(f"- **Accuracy:** `N/A (ไม่มีข้อมูล Ground Truth)`"))
        
    # OCR Test Display
    display(Markdown(f"<hr>"))
    display(Markdown(f"### 📝 ผลลัพธ์ OCR ฉบับเต็ม (เทียบก่อน-หลัง Post-processing)"))

    for index, row in df_results.iterrows():
        display(Markdown(f"---"))
        display(Markdown(f"#### 📄 **ไฟล์:** `{row['File']}`"))

        # Expander for Raw OCR Text
        html_raw = f"""
        <details>
            <summary>คลิกเพื่อดูผลลัพธ์ OCR (Processed Text)</summary>
            <div style="background-color:#f7f7f7; border:1px solid #ddd; padding:10px; margin-top:10px; white-space: pre-wrap; word-wrap: break-word;">{row['Processed OCR Text']}</div>
        </details>
        """
        display(HTML(html_raw))

        # Expander for Processed Text
        html_processed = f"""
        <details>
            <summary>คลิกเพื่อดู Ground Truth Text</summary>
            <div style="background-color:#e6ffed; border:1px solid #b7e1cd; padding:10px; margin-top:5px; white-space: pre-wrap; word-wrap: break-word;">{row['Ground Truth Text']}</div>
        </details>
        """
        display(HTML(html_processed))

### 🚀 เริ่มการทดสอบระบบ OCR กับไฟล์ทั้งหมด 10 ไฟล์

กำลังประมวลผลไฟล์:   0%|          | 0/10 [00:00<?, ?it/s]

   - ✅ พบไฟล์ Ground Truth!
     - SUCCESS: อ่านเนื้อหาสำเร็จ มี 869 ตัวอักษร
   - ✅ พบไฟล์ Ground Truth!
     - SUCCESS: อ่านเนื้อหาสำเร็จ มี 773 ตัวอักษร
   - ✅ พบไฟล์ Ground Truth!
     - SUCCESS: อ่านเนื้อหาสำเร็จ มี 1127 ตัวอักษร
   - ✅ พบไฟล์ Ground Truth!
     - SUCCESS: อ่านเนื้อหาสำเร็จ มี 1338 ตัวอักษร
   - ✅ พบไฟล์ Ground Truth!
     - SUCCESS: อ่านเนื้อหาสำเร็จ มี 846 ตัวอักษร
   - ✅ พบไฟล์ Ground Truth!
     - SUCCESS: อ่านเนื้อหาสำเร็จ มี 1836 ตัวอักษร
   - ✅ พบไฟล์ Ground Truth!
     - SUCCESS: อ่านเนื้อหาสำเร็จ มี 726 ตัวอักษร
   - ✅ พบไฟล์ Ground Truth!
     - SUCCESS: อ่านเนื้อหาสำเร็จ มี 895 ตัวอักษร
   - ✅ พบไฟล์ Ground Truth!
     - SUCCESS: อ่านเนื้อหาสำเร็จ มี 761 ตัวอักษร
   - ✅ พบไฟล์ Ground Truth!
     - SUCCESS: อ่านเนื้อหาสำเร็จ มี 1136 ตัวอักษร


<hr>

### 📊 ตารางสรุปผลการทดสอบ

...

#### 📄 **ไฟล์:** `012-2.pdf`

---

#### 📄 **ไฟล์:** `013-2.pdf`

---

#### 📄 **ไฟล์:** `015-2.pdf`

## RTARF Memo

In [5]:
# Focus on Raw OCR-Public Board

pdf_files = sorted(list(PDF_BOARD_DIR.glob("*.pdf")))

if not pdf_files:
    display(Markdown(f"<font color='red'>**ไม่พบไฟล์ PDF ในโฟลเดอร์ `{PDF_BOARD_DIR}` กรุณาตรวจสอบว่า Path ถูกต้อง**</font>"))
else:
    display(Markdown(f"### 🚀 เริ่มการทดสอบระบบ OCR กับไฟล์ทั้งหมด {len(pdf_files)} ไฟล์"))
    
    results_list = []
    
    for pdf_path in tqdm(pdf_files, desc="กำลังประมวลผลไฟล์"):

        # run OCR
        ocr_result, exec_time = run_typhoon_ocr(pdf_path)
        raw_ocr_text = ocr_result.get("text", "")
        
        # load Ground Truth
        pdf_stem = pdf_path.stem
        gt_path = GROUND_TRUTH_DIR / f"{pdf_stem}.txt"
        print(f"   - PDF Stem: '{pdf_stem}'")
        print(f"   - กำลังมองหา Ground Truth ที่: '{gt_path}'")
        
        ground_truth_text = ""
        if gt_path.is_file():
            print(f"   - ✅ พบไฟล์ Ground Truth!")
            try:
                ground_truth_text = gt_path.read_text(encoding="utf-8").strip()
                if ground_truth_text:
                    print(f"     - SUCCESS: อ่านเนื้อหาสำเร็จ มี {len(ground_truth_text)} ตัวอักษร")
                else:
                    print(f"     - WARNING: อ่านไฟล์สำเร็จ แต่ไฟล์ว่างเปล่า!")
            except Exception as e:
                print(f"     - ❌ ERROR: เกิดข้อผิดพลาดในการอ่านไฟล์: {e}")
        else:
            print(f"   - ❌ ไม่พบไฟล์ Ground Truth")
        
        # Metrics Calculation
        metrics = calculate_metrics(raw_ocr_text, ground_truth_text)

        results_list.append({
            "File": pdf_path.name,
            "Time (s)": exec_time,
            "WER": metrics.get('wer'), 
            "CER": metrics.get('cer'), 
            "Raw OCR Text": raw_ocr_text,
            "Ground Truth Text": ground_truth_text 
        })
        

    df_results = pd.DataFrame(results_list)

    df_results["WAcc"] = 1 - df_results["WER"].fillna(1) # ถ้า WER เป็น None ให้ถือว่า Error 100%
    df_results["CAcc"] = 1 - df_results["CER"].fillna(1) # ถ้า CER เป็น None ให้ถือว่า Error 100%
    display(Markdown(f"<hr>"))
    display(Markdown(f"### 📊 ตารางสรุปผลการทดสอบ (Raw OCR Accuracy)"))

    display_cols = ["File", "Time (s)", "CER", "WER", "CAcc", "WAcc"]

    def safe_percent_format(val):
        if isinstance(val, (int, float)) and pd.notna(val):
            return f"{val:.2%}"
        return "N/A"

    has_numeric_metrics = pd.to_numeric(df_results["CER"], errors='coerce').notna().any()

    styler = df_results[display_cols].style.format({
        "Time (s)": "{:.2f}",
        "CER": safe_percent_format,
        "WER": safe_percent_format,
        "CAcc": safe_percent_format,
        "WAcc": safe_percent_format,
    }).set_properties(**{'text-align': 'left'}).set_table_styles(
        [dict(selector='th', props=[('text-align', 'left')])]
    )

    if has_numeric_metrics:
        styler.background_gradient(cmap='RdYlGn_r', subset=["CER", "WER"])
        styler.background_gradient(cmap='YlGn', subset=["CAcc", "WAcc"])
    
    display(styler)
    display(Markdown(f"### 📈 ค่าเฉลี่ยรวม"))
    
    avg_metrics = df_results[["Time (s)", "CAcc", "WAcc"]].mean()

    display(Markdown(f"- **เวลาประมวลผลเฉลี่ย:** `{avg_metrics['Time (s)']:.2f}` วินาที/ไฟล์"))
    if pd.notna(avg_metrics['CAcc']):
        display(Markdown(f"- **Character Accuracy (CAcc) เฉลี่ย:** `{avg_metrics['CAcc']:.2%}`"))
        display(Markdown(f"- **Word Accuracy (WAcc) เฉลี่ย:** `{avg_metrics['WAcc']:.2%}`"))
    else:
        display(Markdown(f"- **Accuracy:** `N/A (ไม่มีข้อมูล Ground Truth)`"))

    # OCR Result Display
    display(Markdown(f"<hr>"))
    display(Markdown(f"### 📝 ผลลัพธ์เปรียบเทียบ (Raw OCR vs Ground Truth)"))
    
    for _, row in df_results.iterrows():
        display(Markdown(f"---"))
        display(Markdown(f"#### 📄 **ไฟล์:** `{row['File']}`"))

        html_output = f"""
        <details>
            <summary>คลิกเพื่อดูผลลัพธ์ Raw OCR</summary>
            <div style="background-color:#f7f7f7; border:1px solid #ddd; padding:10px; margin-top:10px; white-space: pre-wrap; word-wrap: break-word;">{row['Raw OCR Text']}</div>
        </details>
        <details>
            <summary>คลิกเพื่อดู Ground Truth</summary>
            <div style="background-color:#e6f7ff; border:1px solid #b3e0ff; padding:10px; margin-top:5px; white-space: pre-wrap; word-wrap: break-word;">{row['Ground Truth Text']}</div>
        </details>
        """
        display(HTML(html_output))

### 🚀 เริ่มการทดสอบระบบ OCR กับไฟล์ทั้งหมด 10 ไฟล์

กำลังประมวลผลไฟล์:   0%|          | 0/10 [00:00<?, ?it/s]

   - PDF Stem: '002-2'
   - กำลังมองหา Ground Truth ที่: 'ground_truth/002-2.txt'
   - ✅ พบไฟล์ Ground Truth!
     - SUCCESS: อ่านเนื้อหาสำเร็จ มี 1537 ตัวอักษร
   - PDF Stem: '004-2'
   - กำลังมองหา Ground Truth ที่: 'ground_truth/004-2.txt'
   - ✅ พบไฟล์ Ground Truth!
     - SUCCESS: อ่านเนื้อหาสำเร็จ มี 1468 ตัวอักษร
   - PDF Stem: '006-2'
   - กำลังมองหา Ground Truth ที่: 'ground_truth/006-2.txt'
   - ✅ พบไฟล์ Ground Truth!
     - SUCCESS: อ่านเนื้อหาสำเร็จ มี 1564 ตัวอักษร
   - PDF Stem: '007-2'
   - กำลังมองหา Ground Truth ที่: 'ground_truth/007-2.txt'
   - ✅ พบไฟล์ Ground Truth!
     - SUCCESS: อ่านเนื้อหาสำเร็จ มี 2846 ตัวอักษร
   - PDF Stem: '014-2'
   - กำลังมองหา Ground Truth ที่: 'ground_truth/014-2.txt'
   - ✅ พบไฟล์ Ground Truth!
     - SUCCESS: อ่านเนื้อหาสำเร็จ มี 1835 ตัวอักษร
   - PDF Stem: '016-2'
   - กำลังมองหา Ground Truth ที่: 'ground_truth/016-2.txt'
   - ✅ พบไฟล์ Ground Truth!
     - SUCCESS: อ่านเนื้อหาสำเร็จ มี 1537 ตัวอักษร
   - PDF Stem: '017-2'
   - กำลังม

<hr>

### 📊 ตารางสรุปผลการทดสอบ (Raw OCR Accuracy)

...

#### 📄 **ไฟล์:** `018-2.pdf`

---

#### 📄 **ไฟล์:** `019-2.pdf`

---

#### 📄 **ไฟล์:** `021-2.pdf`

In [6]:
# Focus on Processed OCR-Public Board

pdf_files = sorted(list(PDF_BOARD_DIR.glob("*.pdf")))

if not pdf_files:
    display(Markdown(f"<font color='red'>**ไม่พบไฟล์ PDF ในโฟลเดอร์ `{PDF_BOARD_DIR}` กรุณาตรวจสอบว่า Path ถูกต้อง**</font>"))
else:
    display(Markdown(f"### 🚀 เริ่มการทดสอบระบบ OCR กับไฟล์ทั้งหมด {len(pdf_files)} ไฟล์"))
    
    results_list = []
    
    for pdf_path in tqdm(pdf_files, desc="กำลังประมวลผลไฟล์"):

        # run OCR
        ocr_result, exec_time = run_typhoon_ocr(pdf_path)
        raw_ocr_text = ocr_result.get("text", "")
        
        # do Post-processing
        processed_ocr_text = post_process_ocr(raw_ocr_text)
        
        # load Ground truth
        pdf_stem = pdf_path.stem
        gt_path = GROUND_TRUTH_DIR / f"{pdf_stem}.txt"
        
        ground_truth_text = ""
        if gt_path.is_file(): 
            print(f"   - ✅ พบไฟล์ Ground Truth!")
            try:
                try:
                    ground_truth_text = gt_path.read_text(encoding="utf-8").strip()
                except UnicodeDecodeError:
                    print("     - WARNING: ไม่สามารถอ่านแบบ utf-8 ได้ ลองอ่านแบบ TIS-620")
                    ground_truth_text = gt_path.read_text(encoding="tis-620").strip()

                if ground_truth_text:
                    print(f"     - SUCCESS: อ่านเนื้อหาสำเร็จ มี {len(ground_truth_text)} ตัวอักษร")
                else:
                    print(f"     - WARNING: อ่านไฟล์สำเร็จ แต่ไฟล์ว่างเปล่า!")
            except Exception as e:
                print(f"     - ❌ ERROR: เกิดข้อผิดพลาดในการอ่านไฟล์: {e}")
        else:
            print(f"   - ❌ ไม่พบไฟล์ Ground Truth หรือไม่ใช่ไฟล์ปกติ (อาจเป็นโฟลเดอร์?)")
        
        # Metrics calculation
        metrics_before = calculate_metrics(raw_ocr_text, ground_truth_text)
        metrics_after = calculate_metrics(processed_ocr_text, ground_truth_text)
        
        results_list.append({
            "File": pdf_path.name,
            "Time (s)": exec_time,
            "WER (Before)": metrics_before['wer'],
            "CER (Before)": metrics_before['cer'],
            "WER (After)": metrics_after['wer'],
            "CER (After)": metrics_after['cer'],
            "Raw OCR Text": raw_ocr_text,
            "Processed OCR Text": processed_ocr_text,
            "Ground Truth Text": ground_truth_text
        })
        
    df_results = pd.DataFrame(results_list)
    display(Markdown(f"<hr>"))
    display(Markdown(f"### 📊 ตารางสรุปผลการทดสอบ"))
    df_results["WAcc (After)"] = 1 - df_results["WER (After)"].fillna(0)
    df_results["CAcc (After)"] = 1 - df_results["CER (After)"].fillna(0)

    display_cols = [
        "File", "Time (s)", 
        "CER (After)", "WER (After)", 
        "CAcc (After)", "WAcc (After)"
    ]

    has_numeric_metrics = pd.to_numeric(df_results["CER (After)"], errors='coerce').notna().any()

    # Format Data 
    styler = df_results[display_cols].style.format({
        "Time (s)": "{:.2f}",
        "CER (After)": safe_percent_format,
        "WER (After)": safe_percent_format,
        "CAcc (After)": safe_percent_format, 
        "WAcc (After)": safe_percent_format, 
    }).background_gradient(
        cmap='RdYlGn_r', 
        subset=["CER (After)", "WER (After)"]
    ).background_gradient(
        cmap='YlGn', 
        subset=["CAcc (After)", "WAcc (After)"]
    ).set_properties(**{'text-align': 'left'}).set_table_styles(
        [dict(selector='th', props=[('text-align', 'left')])]
    )

    display(styler)
    display(Markdown(f"### 📈 ค่าเฉลี่ยรวม"))

    avg_metrics = df_results[display_cols[1:]].mean() 
    display(Markdown(f"- **เวลาประมวลผลเฉลี่ย:** `{avg_metrics['Time (s)']:.2f}` วินาที/ไฟล์"))

    if pd.notna(avg_metrics['CAcc (After)']):
        display(Markdown(f"- **Character Accuracy (CAcc) เฉลี่ย:** `{avg_metrics['CAcc (After)']:.2%}` (ความถูกต้องระดับตัวอักษร)"))
        display(Markdown(f"- **Word Accuracy (WAcc) เฉลี่ย:** `{avg_metrics['WAcc (After)']:.2%}` (ความถูกต้องระดับคำ)"))
    else:
        display(Markdown(f"- **Accuracy:** `N/A (ไม่มีข้อมูล Ground Truth)`"))
        
    # OCR Test Display
    display(Markdown(f"<hr>"))
    display(Markdown(f"### 📝 ผลลัพธ์ OCR ฉบับเต็ม (เทียบก่อน-หลัง Post-processing)"))

    for index, row in df_results.iterrows():
        display(Markdown(f"---"))
        display(Markdown(f"#### 📄 **ไฟล์:** `{row['File']}`"))

        # Expander for Raw OCR Text
        html_raw = f"""
        <details>
            <summary>คลิกเพื่อดูผลลัพธ์ OCR (Processed Text)</summary>
            <div style="background-color:#f7f7f7; border:1px solid #ddd; padding:10px; margin-top:10px; white-space: pre-wrap; word-wrap: break-word;">{row['Processed OCR Text']}</div>
        </details>
        """
        display(HTML(html_raw))

        # Expander for Processed Text
        html_processed = f"""
        <details>
            <summary>คลิกเพื่อดู Ground Truth Text</summary>
            <div style="background-color:#e6ffed; border:1px solid #b7e1cd; padding:10px; margin-top:5px; white-space: pre-wrap; word-wrap: break-word;">{row['Ground Truth Text']}</div>
        </details>
        """
        display(HTML(html_processed))

### 🚀 เริ่มการทดสอบระบบ OCR กับไฟล์ทั้งหมด 10 ไฟล์

กำลังประมวลผลไฟล์:   0%|          | 0/10 [00:00<?, ?it/s]

   - ✅ พบไฟล์ Ground Truth!
     - SUCCESS: อ่านเนื้อหาสำเร็จ มี 1537 ตัวอักษร
   - ✅ พบไฟล์ Ground Truth!
     - SUCCESS: อ่านเนื้อหาสำเร็จ มี 1468 ตัวอักษร
   - ✅ พบไฟล์ Ground Truth!
     - SUCCESS: อ่านเนื้อหาสำเร็จ มี 1564 ตัวอักษร
   - ✅ พบไฟล์ Ground Truth!
     - SUCCESS: อ่านเนื้อหาสำเร็จ มี 2846 ตัวอักษร
   - ✅ พบไฟล์ Ground Truth!
     - SUCCESS: อ่านเนื้อหาสำเร็จ มี 1835 ตัวอักษร
   - ✅ พบไฟล์ Ground Truth!
     - SUCCESS: อ่านเนื้อหาสำเร็จ มี 1537 ตัวอักษร
   - ✅ พบไฟล์ Ground Truth!
     - SUCCESS: อ่านเนื้อหาสำเร็จ มี 1539 ตัวอักษร
   - ✅ พบไฟล์ Ground Truth!
     - SUCCESS: อ่านเนื้อหาสำเร็จ มี 1364 ตัวอักษร
   - ✅ พบไฟล์ Ground Truth!
     - SUCCESS: อ่านเนื้อหาสำเร็จ มี 2069 ตัวอักษร
   - ✅ พบไฟล์ Ground Truth!
     - SUCCESS: อ่านเนื้อหาสำเร็จ มี 1612 ตัวอักษร


<hr>

### 📊 ตารางสรุปผลการทดสอบ

...

#### 📄 **ไฟล์:** `018-2.pdf`

---

#### 📄 **ไฟล์:** `019-2.pdf`

---

#### 📄 **ไฟล์:** `021-2.pdf`